In [ ]:
#Entrenamiento
import pandas as pd

Path = "/content/drive/MyDrive/PCś/RP/"
dfW = pd.read_csv(Path+'Weekly.csv')
dfW['Direction2'] = dfW['Direction'].factorize()[0]

#Conjunto de entrenamiento
dfW_train = dfW[dfW.Year <= 2008]

#Conjunto de pruebas
dfW_test = dfW[dfW.Year >= 2009]
X_test  = dfW_test[['Lag1','Lag2','Lag3','Lag4','Lag5','Volume']]
y_test  = dfW_test[['Direction']]
y_testSKL = np.asarray(dfW_test['Direction2'])

#Se utiliza el método binomial de statsmodel y se entrena el modelo
import statsmodels.api as sm
import statsmodels.formula.api as smf
mod_bn2 = smf.glm(formula='Direction~Lag2', data=dfW_train, family=sm.families.Binomial()).fit()
print(mod_bn2.summary())

#Se agregan constantes al conjunto de prueba para usar stastsmodel
X_test = sm.add_constant(X_test)
#Se realizan predicciones con el modelo
y_prob = mod_bn2.predict(X_test)

# reset indices
y_test = y_test.reset_index()['Direction']
y_prob = y_prob.reset_index()[0]

decision_prob = 0.5 #Frontera de decisión
df_ = pd.DataFrame({'True default status': y_test,
                    'Predicted default status': y_prob > decision_prob})

# matriz de confusión 
import numpy as np

df_.replace(to_replace={True:1, False:0}, inplace=True) #Se remplaza los true por 1 y los false por 0
y_pred = np.asarray(df_['Predicted default status'])
df_.replace(to_replace={0:'Up', 1:'Down'}, inplace=True)
confusion_matrix = df_.groupby(['Predicted default status','True default status']).size().unstack('True default status')
print('\n',confusion_matrix)
VerifAccur =  (5+34)/104

#Metricas de sklearn
import sklearn.linear_model as lm
import sklearn.discriminant_analysis as da
import sklearn.neighbors as nb
from sklearn.metrics import confusion_matrix, classification_report, precision_score, accuracy_score

Preci = precision_score(y_testSKL, y_pred)
Acc   = accuracy_score(y_testSKL, y_pred)

#out
print('\nprecision_score: ',Preci)
print('accuracy_score:  ',Acc)
print('Verific_Accur:   ',VerifAccur)